### scrapy
    비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠름

In [ ]:
# items.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할때 headers 정보와 같은 내용을 설정
# pipelines.py : 데이터를 수집한 후에 코드 실행 정의
# settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙
# spiders : 디렉토리 : 크롤링 절차 정의

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [2]:
#  g마켓 베스트 200 상품 데이터 수집

In [3]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\code\크롤링\day1\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [ ]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터

In [6]:
# 링크데이터 수집
request = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(request.url, body = request.text, encoding ="utf-8")

In [11]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/div[1]/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953127243935855',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953127243935855']

In [12]:
# 상세페이지 데이터 수집

In [13]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body = request.text, encoding ="utf-8")

In [15]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [16]:
#3. items.py : 코드 작성 : model(데이터의 구조 : 수집할 데이터의 컬럼을 정의)

In [17]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py 코드 작성

In [18]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [20]:
# 5.스크레피 프로젝트 실행
# 터미널에서 gmarket 디렉토리 -> 
# scrapy crawl GMB -o items.csv 저장

In [21]:
%pwd

'C:\\Users\\User\\Desktop\\code\\크롤링\\day1'

In [22]:
pd.read_csv("gmarket/items.csv").tail(2)

,link,price,title
18,http://item.gmarket.co.kr/Item?goodscode=22464...,"19,360",서울우유 클릭유 커피 200ml x 24입 (1박스)
19,http://item.gmarket.co.kr/Item?goodscode=20965...,"109,000",BLDC 입체회전 써큘레이터 SIF-DPNW90 ---시즌오프 특가---
